### stage_model_classifier

prediction models of sign classifiers on stage dataset

data format:
    target , eventid ,    car_number,    stageid,     features...

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# to use only one GPU.
# use this on r-001
# otherwise comment
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb


In [3]:
# bulid regression model
classifiers = ['currank','avgrank','dice','lr','lrl1','lsvc','lsvcl2','rf','lrbias','xgb']
def get_classifier(classifier = 'lr'):
    
    class_weight = None
    
    if classifier == "lsvc":
        clf = LinearSVC(penalty='l1',dual=False, tol=1e-3, class_weight=class_weight )
    elif classifier == "lsvcl2":
        clf = LinearSVC(penalty='l2', tol=1e-4, class_weight=class_weight)
    elif classifier == 'rf':
        #clf = RandomForestClassifier(n_estimators=100, n_jobs=4,criterion='entropy', min_samples_split=1,class_weight = class_weight)
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1,criterion='entropy', class_weight = class_weight)
    elif classifier == 'lr':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = False, verbose = 0)
    elif classifier == 'lrbias':
        clf = LogisticRegression(class_weight = class_weight, n_jobs=-1, fit_intercept = True, verbose = 1)
    elif classifier == 'lrl1':
        clf = LogisticRegression(class_weight = class_weight, penalty='l1',n_jobs=-1)
    elif classifier == 'xgb':
        clf = xgb.XGBClassifier(booster = 'gbtree', nthread = -1, subsample = 1, 
                                n_estimators = 600, colsample_bytree = 1, max_depth = 6, min_child_weight = 1)
    elif classifier == 'dice':
        clf = RandomDice('1234')
    elif classifier == 'currank':
        clf = CurRank()
    elif classifier == 'avgrank':
        clf = AverageRank()        
    else:
        clf = None
        
    return clf


class CurRank():
    """
    predict with current rank
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = [0 for x in range(test_x.shape[0])]
        return np.array(pred_y)
    
class AverageRank():
    """
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    """
    def __init__(self):
        pass
    def fit(self, x, y):
        pass
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            #13, change_in_rank_all
            pred_y.append(x[13])
        pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in pred_y])
        return np.array(pred_y_avg)   

class RandomDice():
    """
    a random dice model
    """
    def __init__(self, seed='1234'):
        self.dist = []
        self.val = []
        random.seed(seed)
    
    def fit(self, x, y):
        total = y.shape[0]
        yval = set(y)
        
        ratio = 0.
        for val in yval:
            self.val.append(val)
            ratio += np.sum(y==val)*1.0 / total
            self.dist.append(ratio)
            
    def predict(self, test_x):
        pred_y = []
        for x in test_x:
            dice = random.random()
            #search in self.dist
            find_idx = -1
            for idx, ratio in enumerate(self.dist):
                if dice <= ratio:
                    find_idx = idx
                    break
            
            #or the last one match
            pred_y.append(self.val[find_idx])
            
        return np.array(pred_y)

def evaluate(test_y, pred_y):
    precision = metrics.precision_score(test_y, pred_y, average=None) 
    recall = metrics.recall_score(test_y, pred_y, average=None)
    f1 = metrics.f1_score(test_y, pred_y, average=None)
    accuracy = metrics.accuracy_score(test_y, pred_y)
    print('precision=%s, recall=%s, f1=%s, accuracy=%.2f'%(precision,recall, f1, accuracy))
    return accuracy
    
#
#features
#    cols=[Myidx, 'target','eventid','car_number','stageid',
#             'firststage','pit_in_caution','start_position',
#             'start_rank','start_rank_ratio','top_pack','bottom_pack',
#             'average_rank','average_rank_all',
#             'change_in_rank','change_in_rank_all','rate_of_change','rate_of_change_all']    
def split_by_eventid(stagedata, eventid):
    """
    split by eventid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['eventid'] != eventid].to_numpy()
    test  = stagedata[stagedata['eventid'] == eventid].to_numpy()

    #2:car_number
    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


def split_by_stageid(stagedata, stageid):
    """
    split by stageid
    """
    #if not eventid in stagedata:
    #    print('error, %d not found in stagedata'%eventid)
    #    return
    
    train = stagedata[stagedata['stageid'] <= stageid].to_numpy()
    test  = stagedata[stagedata['stageid'] > stageid].to_numpy()

    train_x = train[:,2:]
    train_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in train[:,1]])
    test_x = test[:,2:]
    test_y = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in test[:,1]])
    
    return train, test, train_x, train_y, test_x, test_y


### baseline
def baseline_model():
    #1. predict with current rank, rankchg = 0
    print('[*] predict with current rank, rankchg = 0')
    pred_y_simple = np.zeros_like(test_y)
    score1 = evaluate(test_y, pred_y_simple)

    #2. predict with average rankchg (change_in_rank_all):idx = 15
    print('[*] predict with average rankchg (change_in_rank_all):idx = 15')
    change_in_rank_all = test[:,15]
    pred_y_avg = np.array([1 if x > 0 else (-1 if x < 0 else 0) for x in change_in_rank_all])
    score2 = evaluate(test_y, pred_y_avg)
    return score1, score2

def classifier_model(name='lr'):
    ### test learning models
    print('[*] predict with %s model'%name)
    clf = get_classifier(name)
    clf.fit(train_x, train_y)

    pred_y = clf.predict(test_x)
    score = evaluate(test_y, pred_y)
    return score

In [4]:
#load data
suffix='-withneighbor'
stagedata = pd.read_csv('stage-2018%s.csv'%suffix)
stagedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 24 columns):
Unnamed: 0                   805 non-null int64
target                       805 non-null int64
eventid                      805 non-null int64
car_number                   805 non-null int64
stageid                      805 non-null int64
firststage                   805 non-null int64
pit_in_caution               805 non-null int64
start_position               805 non-null int64
start_rank                   805 non-null int64
start_rank_ratio             805 non-null float64
top_pack                     805 non-null int64
bottom_pack                  805 non-null int64
average_rank                 805 non-null float64
average_rank_all             805 non-null float64
change_in_rank               805 non-null int64
change_in_rank_all           805 non-null float64
rate_of_change               805 non-null int64
rate_of_change_all           805 non-null float64
prev_nb0_change_in_ra

In [5]:
stagedata.head(5)

,Unnamed: 0,target,eventid,car_number,stageid,firststage,pit_in_caution,start_position,start_rank,start_rank_ratio,...,change_in_rank,change_in_rank_all,rate_of_change,rate_of_change_all,prev_nb0_change_in_rank,prev_nb1_change_in_rank,prev_nb2_change_in_rank,follow_nb0_change_in_rank,follow_nb1_change_in_rank,follow_nb2_change_in_rank
0,0,0,0,1,0,0,0,7,7,0.304348,...,0,0.000000,0,0.000000,0,0,0,0,0,0
1,1,-5,0,1,1,1,1,7,7,0.304348,...,0,0.000000,0,0.000000,-2,-1,1,3,0,-1
2,2,3,0,1,2,1,0,7,2,0.086957,...,-5,-2.500000,0,0.000000,1,0,0,-5,-5,1
3,3,-4,0,1,3,1,0,7,5,0.217391,...,3,-0.666667,8,1.500000,-1,-2,-3,-6,-8,-5
4,4,0,0,1,4,1,1,7,1,0.043478,...,-4,-1.500000,-7,-1.333333,0,0,0,-1,-2,-3


In [6]:
cols = ['runid','trainsize','testsize','testdistribution']
cols.extend(classifiers)
print('cols:%s'%cols)
retdf = pd.DataFrame([],columns=cols)

eventsname = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
events = set(stagedata['eventid'])
for eventid in events:
    print('Testset = %s'%eventsname[eventid])
    
    train, test, train_x, train_y, test_x, test_y = split_by_eventid(stagedata, eventid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = [eventsname[eventid],train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])        
    
retdf.to_csv('crossvalid_stagedata_splitbyevent%s.csv'%suffix)
df_event = retdf

cols:['runid', 'trainsize', 'testsize', 'testdistribution', 'currank', 'avgrank', 'dice', 'lr', 'lrl1', 'lsvc', 'lsvcl2', 'rf', 'lrbias', 'xgb']
Testset = Phoenix
[*] predict with currank model
precision=[0.         0.14035088 0.        ], recall=[0. 1. 0.], f1=[0.         0.24615385 0.        ], accuracy=0.14
[*] predict with avgrank model
precision=[0.48275862 0.16129032 0.16      ], recall=[0.46666667 0.3125     0.10526316], f1=[0.47457627 0.21276596 0.12698413], accuracy=0.32
[*] predict with dice model
precision=[0.52173913 0.2        0.31578947], recall=[0.4        0.375      0.31578947], f1=[0.45283019 0.26086957 0.31578947], accuracy=0.37
[*] predict with lr model
precision=[0.61971831 0.22222222 0.47058824], recall=[0.73333333 0.125      0.42105263], f1=[0.67175573 0.16       0.44444444], accuracy=0.54
[*] predict with lrl1 model
precision=[0.625      0.22222222 0.48484848], recall=[0.75       0.125      0.42105263], f1=[0.68181818 0.16       0.45070423], accuracy=0.55
[*] pre

/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.609375   0.15384615 0.43243243], recall=[0.65       0.125      0.42105263], f1=[0.62903226 0.13793103 0.42666667], accuracy=0.50
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.6440678  0.21428571 0.69230769], recall=[0.63333333 0.5625     0.23684211], f1=[0.63865546 0.31034483 0.35294118], accuracy=0.49
[*] predict with rf model
precision=[0.59375    0.26666667 0.48571429], recall=[0.63333333 0.25       0.44736842], f1=[0.61290323 0.25806452 0.46575342], accuracy=0.52
[*] predict with lrbias model
[LibLinear]precision=[0.62857143 0.2        0.47058824], recall=[0.73333333 0.125      0.42105263], f1=[0.67692308 0.15384615 0.44444444], accuracy=0.54
[*] predict with xgb model


...

precision=[0.85714286 0.35135135 0.4375    ], recall=[0.28571429 0.92857143 0.20588235], f1=[0.42857143 0.50980392 0.28      ], accuracy=0.43
[*] predict with lsvc model
precision=[0.8        0.35135135 0.46666667], recall=[0.28571429 0.92857143 0.20588235], f1=[0.42105263 0.50980392 0.28571429], accuracy=0.43
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


precision=[0.59090909 0.30263158 0.66666667], recall=[0.30952381 0.82142857 0.11764706], f1=[0.40625    0.44230769 0.2       ], accuracy=0.38
[*] predict with rf model
precision=[0.58333333 0.3902439  0.48148148], recall=[0.5        0.57142857 0.38235294], f1=[0.53846154 0.46376812 0.42622951], accuracy=0.48
[*] predict with lrbias model
[LibLinear]precision=[0.8        0.36619718 0.5       ], recall=[0.28571429 0.92857143 0.26470588], f1=[0.42105263 0.52525253 0.34615385], accuracy=0.45
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.43333333 0.4047619  0.375     ], recall=[0.30952381 0.60714286 0.35294118], f1=[0.36111111 0.48571429 0.36363636], accuracy=0.40
rec:['Gateway', 701, 104, '+:34,0:28,-:42', 0.2692307692307692, 0.3269230769230769, 0.25, 0.4807692307692308, 0.4326923076923077, 0.4326923076923077, 0.38461538461538464, 0.4807692307692308, 0.4519230769230769, 0.40384615384615385]


In [7]:
df_event

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,Phoenix,691,114,"+:38,0:16,-:60",0.140351,0.324561,0.368421,0.543860,0.552632,0.500000,0.491228,0.517544,0.543860,0.438596
0,Indy500,580,225,"+:82,0:47,-:96",0.208889,0.324444,0.395556,0.560000,0.555556,0.555556,0.542222,0.520000,0.560000,0.528889
0,Texas,678,127,"+:39,0:34,-:54",0.267717,0.314961,0.291339,0.559055,0.574803,0.598425,0.559055,0.503937,0.574803,0.448819
0,Iowa,696,109,"+:42,0:28,-:39",0.256881,0.293578,0.256881,0.348624,0.357798,0.357798,0.431193,0.394495,0.357798,0.302752
0,Pocono,679,126,"+:29,0:61,-:36",0.484127,0.277778,0.238095,0.365079,0.365079,0.349206,0.468254,0.333333,0.357143,0.460317
0,Gateway,701,104,"+:34,0:28,-:42",0.269231,0.326923,0.250000,0.480769,0.432692,0.432692,0.384615,0.480769,0.451923,0.403846


In [8]:
retdf = pd.DataFrame([],columns=cols)

for stageid in range(8):
    train, test, train_x, train_y, test_x, test_y =split_by_stageid(stagedata, stageid)
    test_distribution = '+:%d,0:%d,-:%d'%(np.sum(test_y>0),np.sum(test_y==0),np.sum(test_y<0))
    #print('Testset by stageid= %s, trainsize=%d, testsize=%d, dist=%s'%
    #      (stageid, train_x.shape[0], test_x.shape[0], test_distribution))
    
    #record
    rec = ['stage%d'%stageid,train_x.shape[0],test_x.shape[0],test_distribution]
    
    acc = [0 for x in range(len(classifiers))]
    for idx, clf in enumerate(classifiers):
        acc[idx] = classifier_model(clf)

    rec.extend(acc)
    print('rec:%s'%rec)
    
    #new df
    df = pd.DataFrame([rec],columns=cols)
    retdf = pd.concat([retdf, df])  
    
retdf.to_csv('crossvalid_stagedata_splitbystage%s.csv'%suffix)
df_stage = retdf

[*] predict with currank model
precision=[0.         0.25613497 0.        ], recall=[0. 1. 0.], f1=[0.         0.40781441 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.37172775 0.22535211 0.23115578], recall=[0.53787879 0.09580838 0.2081448 ], f1=[0.43962848 0.13445378 0.21904762], accuracy=0.31
[*] predict with dice model
precision=[0.41197183 0.25806452 0.3956044 ], recall=[0.44318182 0.28742515 0.32579186], f1=[0.4270073  0.27195467 0.3573201 ], accuracy=0.36
[*] predict with lr model
precision=[0.49633252 0.36842105 0.56603774], recall=[0.76893939 0.41916168 0.13574661], f1=[0.60326895 0.39215686 0.2189781 ], accuracy=0.46
[*] predict with lrl1 model
precision=[0.49867374 0.37209302 0.4       ], recall=[0.71212121 0.47904192 0.10859729], f1=[0.58658346 0.41884817 0.17081851], accuracy=0.45
[*] predict with lsvc model
precision=[0.51734104 0.35341365 0.36842105], recall=[0.6780303  0.52694611 0.09502262], f1=[0.58688525 0.42307692 0.15107914], accuracy=0.44


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", 

precision=[0.46644295 0.33333333 0.35947712], recall=[0.52651515 0.4011976  0.24886878], f1=[0.49466192 0.36413043 0.29411765], accuracy=0.40
[*] predict with lrbias model
[LibLinear]precision=[0.49234694 0.35576923 0.48076923], recall=[0.73106061 0.44311377 0.11312217], f1=[0.58841463 0.39466667 0.18315018], accuracy=0.45
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.4375     0.30677291 0.29655172], recall=[0.42424242 0.46107784 0.19457014], f1=[0.43076923 0.36842105 0.23497268], accuracy=0.36
rec:['stage0', 153, 652, '+:221,0:167,-:264', 0.2561349693251534, 0.3128834355828221, 0.36349693251533743, 0.4647239263803681, 0.44785276073619634, 0.44171779141104295, 0.2745398773006135, 0.4003067484662577, 0.44785276073619634, 0.3558282208588957]
[*] predict with currank model
precision=[0.         0.26305609 0.        ], recall=[0. 1. 0.], f1=[0.         0.41653905 0.        ], accuracy=0.26
[*] predict with avgrank model
precision=[0.34674923 0.24242424 0.24223602], recall=[0.57435897 0.05882353 0.20967742], f1=[0.43243243 0.09467456 0.22478386], accuracy=0.31
[*] predict with dice model
precision=[0.39312977 0.22881356 0.37956204], recall=[0.52820513 0.19852941 0.27956989], f1=[0.45076586 0.21259843 0.32198142], accuracy=0.35
[*] predict with lr model
precision=[0.42384106 0.35772358 0.57608696], recall=[0.65641026 0.32352941 0.28494624], f

...

precision=[1.  0.3 0. ], recall=[0.375 0.75  0.   ], f1=[0.54545455 0.42857143 0.        ], accuracy=0.38
[*] predict with lsvc model
precision=[1.   0.25 0.  ], recall=[0.25 0.75 0.  ], f1=[0.4   0.375 0.   ], accuracy=0.31
[*] predict with lsvcl2 model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


precision=[0.   0.25 0.  ], recall=[0. 1. 0.], f1=[0.  0.4 0. ], accuracy=0.25
[*] predict with rf model
precision=[0.7 0.5 1. ], recall=[0.875 0.5   0.5  ], f1=[0.77777778 0.5        0.66666667], accuracy=0.69
[*] predict with lrbias model
[LibLinear]precision=[1.         0.27272727 0.        ], recall=[0.25 0.75 0.  ], f1=[0.4 0.4 0. ], accuracy=0.31
[*] predict with xgb model


/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/scratch/hpda/anaconda3/envs/predictor/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 48.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


precision=[0.71428571 0.5        0.6       ], recall=[0.625 0.5   0.75 ], f1=[0.66666667 0.5        0.66666667], accuracy=0.62
rec:['stage7', 789, 16, '+:4,0:4,-:8', 0.25, 0.1875, 0.3125, 0.3125, 0.375, 0.3125, 0.25, 0.6875, 0.3125, 0.625]


In [9]:
#xgb max_tree_depth=3
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:221,0:167,-:264",0.256135,0.312883,0.363497,0.464724,0.447853,0.441718,0.274540,0.400307,0.447853,0.355828
0,stage1,288,517,"+:186,0:136,-:195",0.263056,0.307544,0.352031,0.435203,0.448743,0.456480,0.450677,0.454545,0.433269,0.462282
0,stage2,421,384,"+:140,0:112,-:132",0.291667,0.263021,0.317708,0.442708,0.427083,0.455729,0.421875,0.447917,0.442708,0.434896
0,stage3,547,258,"+:91,0:89,-:78",0.344961,0.228682,0.333333,0.457364,0.441860,0.445736,0.406977,0.453488,0.457364,0.441860
0,stage4,657,148,"+:48,0:53,-:47",0.358108,0.209459,0.344595,0.500000,0.500000,0.466216,0.425676,0.466216,0.506757,0.445946
0,stage5,725,80,"+:26,0:29,-:25",0.362500,0.200000,0.325000,0.575000,0.587500,0.550000,0.537500,0.575000,0.587500,0.612500
0,stage6,767,38,"+:11,0:13,-:14",0.342105,0.236842,0.263158,0.447368,0.447368,0.473684,0.368421,0.500000,0.473684,0.526316
0,stage7,789,16,"+:4,0:4,-:8",0.250000,0.187500,0.312500,0.312500,0.375000,0.312500,0.250000,0.687500,0.312500,0.625000


In [10]:
#xgb max_tree_depth=6
df_stage

,runid,trainsize,testsize,testdistribution,currank,avgrank,dice,lr,lrl1,lsvc,lsvcl2,rf,lrbias,xgb
0,stage0,153,652,"+:221,0:167,-:264",0.256135,0.312883,0.363497,0.464724,0.447853,0.441718,0.274540,0.400307,0.447853,0.355828
0,stage1,288,517,"+:186,0:136,-:195",0.263056,0.307544,0.352031,0.435203,0.448743,0.456480,0.450677,0.454545,0.433269,0.462282
0,stage2,421,384,"+:140,0:112,-:132",0.291667,0.263021,0.317708,0.442708,0.427083,0.455729,0.421875,0.447917,0.442708,0.434896
0,stage3,547,258,"+:91,0:89,-:78",0.344961,0.228682,0.333333,0.457364,0.441860,0.445736,0.406977,0.453488,0.457364,0.441860
0,stage4,657,148,"+:48,0:53,-:47",0.358108,0.209459,0.344595,0.500000,0.500000,0.466216,0.425676,0.466216,0.506757,0.445946
0,stage5,725,80,"+:26,0:29,-:25",0.362500,0.200000,0.325000,0.575000,0.587500,0.550000,0.537500,0.575000,0.587500,0.612500
0,stage6,767,38,"+:11,0:13,-:14",0.342105,0.236842,0.263158,0.447368,0.447368,0.473684,0.368421,0.500000,0.473684,0.526316
0,stage7,789,16,"+:4,0:4,-:8",0.250000,0.187500,0.312500,0.312500,0.375000,0.312500,0.250000,0.687500,0.312500,0.625000
